In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
import h5py
import numpy as np
import torchvision.transforms as transforms
import pickle as pkl
import random
import math
transform_test = transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))]
        )

class get_dataset(Dataset):
    def __init__(self, h5_file_path):    
        self.h5_file_path = h5_file_path
        with h5py.File(self.h5_file_path, 'r') as data_file:
            self.images = np.array(data_file['data']) 
            self.labels = np.array(data_file['label']) 
           
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, item):
        
        label = torch.tensor(self.labels[item])
        image = np.array(self.images[item, :, :, :])
        image = np.transpose(image, (1, 2, 0))  
        image = transform_test(image)
    
        return [image, label]
 

In [ ]:
from sklearn.metrics import roc_curve,auc
from scipy.spatial import distance
import numpy as np
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr, spearmanr
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset, TensorDataset
from torch.utils.data import Subset
import torchvision

BATCH_SIZE = 10

def calculate_auc(list_a, list_b):
    l1,l2 = len(list_a),len(list_b)
    y_true,y_score = [],[]
    for i in range(l1):
        y_true.append(0)
    for i in range(l2):
        y_true.append(1)
    y_score.extend(list_a)
    y_score.extend(list_b)
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    return auc(fpr, tpr)


class FeatureHook():

    def __init__(self, module):
        self.hook = module.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output):
        self.output = output
    def close(self):
        self.hook.remove()



def correlation(m,n):
    m = F.normalize(m,dim=-1)
    n = F.normalize(n,dim=-1).transpose(0,1)
    cose = torch.mm(m,n)
    matrix = 1-cose
    matrix = matrix/2
    return matrix

def pairwise_euclid_distance(A):
    sqr_norm_A = torch.unsqueeze(torch.sum(torch.pow(A, 2),dim=1),dim=0)
    sqr_norm_B = torch.unsqueeze(torch.sum(torch.pow(A, 2), dim=1), dim=1)
    inner_prod = torch.matmul(A, A.transpose(0,1))
    tile1 = torch.reshape(sqr_norm_A,[A.shape[0],1])
    tile2 = torch.reshape(sqr_norm_B,[1,A.shape[0]])
    return tile1+tile2 - 2*inner_prod


def correlation_dist(A):
    A = F.normalize(A,dim=-1)
    cor = pairwise_euclid_distance(A)
    cor = torch.exp(-cor)

    return cor


def cal_cor(model,dataloader):
    model.eval()
    model = model.cuda()
    outputs = []
    with torch.no_grad():
        for i,(x,y) in enumerate(dataloader):
            x, y = x.float().cuda(), y.cuda()
            output = model(x.float())
            outputs.append(output.cpu().detach())

    output = torch.cat(outputs,dim=0)
    cor_mat = correlation(output,output)


    model = model.cpu()
    return cor_mat

def output_to_label(output):
    shape = output.shape
    pred = torch.argmax(output,dim=1)

    preds = 0.01 * torch.ones(shape)

    for i in range(shape[0]):
        preds[i,pred[i]]=1

    preds = torch.softmax(preds,dim=-1)

    print(preds[0,:])
    return preds

def cal_cor_onehot(model, dataloader):
    model.eval()
    model = model.cuda()
    preds = []
    for x,y in dataloader:
        x = x.float().cuda()
        output = model(x)
        pred = torch.argmax(output, dim=1)
        preds.append(pred.cpu().detach())
    preds = torch.cat(preds, dim=0)
    model = model.cpu()
    return preds

#unic-hard label, Sim
def cal_acc(models, i):
   
    cor_mats = []

    train_data = get_dataset('./data/generate_DiffFP_boundary_100.h5')
    wrong_indices = list(range(0, 200, 2))  
    train_data_subset = Subset(train_data, wrong_indices)
    train_loader = DataLoader(train_data_subset, shuffle=False, batch_size=BATCH_SIZE)
#
    for i in range(len(models)):
        model = models[i]
        cor_mat = cal_cor_onehot(model, train_loader)
        cor_mats.append(cor_mat)

    print(len(cor_mats), cor_mat.shape)

    baseline = cor_mats[0]

    acc = []

    for i in range(1, len(cor_mats)):

        diff_count = (cor_mats[i] == baseline).sum().item()
        acc.append(diff_count/100) 


    print(acc)
  
#Tamper detection rate
def cal_fragility(models, i):
    cor_mats = []
    train_data = get_dataset('./data/generate_DiffFP_boundary_100.h5') 
    wrong_indices = list(range(0, 200, 2)) 
    train_data_subset = Subset(train_data, wrong_indices)
    train_loader = DataLoader(train_data_subset, shuffle=False, batch_size=BATCH_SIZE)   
    for i in range(len(models)):
        model = models[i]
        cor_mat = cal_cor_onehot(model, train_loader)
        cor_mats.append(cor_mat)

    print(len(cor_mats), cor_mat.shape)
    
    baseline = cor_mats[0]
    differences = []
    
    for i in range(1, len(cor_mats)):
        diff_count = (cor_mats[i] != baseline).sum().item()
        differences.append(diff_count)
        print(f"{i}:{diff_count}, diff rate:{diff_count/100}")
    
#AUC
def cal_correlation(models,i):

    cor_mats = []


    train_data =get_dataset('./data/generate_DiffFP_boundary_100.h5')
    indices =  list(range(0, 200, 2)) #wrong 
    train_data_subset = Subset(train_data, indices)
    train_loader = DataLoader(train_data_subset, shuffle=False, batch_size=BATCH_SIZE)



    for i in range(len(models)):

        model = models[i]
        cor_mat = cal_cor(model, train_loader)
        cor_mats.append(cor_mat)

    print(len(cor_mats), cor_mat.shape)
    


    diff = torch.zeros(len(models))
    for i in range(len(models) - 1):
        iter = i + 1
        diff[i] = torch.sum(torch.abs(cor_mats[iter] - cor_mats[0])) / (cor_mat.shape[0] * cor_mat.shape[1]) #L1
    print(cor_mat.shape[0],cor_mat.shape[1])

    list1 = diff[:20]
    list2 = diff[20:40]
    list3 = diff[40:60]
    list4 = diff[60:65]
    list5 = diff[65:80]
    list6_1 = diff[80:90]
    list6_2 = diff[90:100]

    auc_p = calculate_auc(list1, list3)
    auc_l = calculate_auc(list2, list3)
    auc_tl = calculate_auc(list4, list3)
    auc_finetune_all = calculate_auc(list6_1, list3)
    auc_finetune_last = calculate_auc(list6_2, list3)
    auc_prune = calculate_auc(list5, list3)

    print("AUC_Prune:", auc_prune)

    print("AUC_MEP:",auc_p,"AUC_MEL:", auc_l)#,
    print("AUC_Finetune_all:",auc_finetune_all, "auc_finetune_last:", auc_finetune_last)
    print("AUC_TL:", auc_tl)
    ave = (auc_prune + auc_tl + auc_finetune_all + auc_finetune_last + auc_p + auc_l) / 6.0
    print(ave)
#  

In [ ]:
import torchvision.models as models
from torch import nn

def reset(cls):
    if cls == 'resnet':
        model = models.resnet18(pretrained=False)
        model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        in_feature = model.fc.in_features
        model.fc = torch.nn.Linear(in_feature, 10)
    elif cls == "vgg":
        model = models.vgg13(pretrained=False)
        model.features[0] = torch.nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
        for i, layer in enumerate(model.features):
            if isinstance(layer, nn.MaxPool2d):
                model.features[i] = nn.MaxPool2d(kernel_size=2, stride=1, padding=1)
        in_feature = model.classifier[-1].in_features
        model.classifier[-1] = torch.nn.Linear(in_feature, 10)

    elif cls == 'dense':
        model = models.densenet121(pretrained=False)
        model.features.conv0 = torch.nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        in_feature = model.classifier.in_features
        model.classifier = torch.nn.Linear(in_feature, 10)

    elif cls == 'mobile':
        model = models.mobilenet_v2(pretrained=False)
        model.features[0][0] = torch.nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False)
        in_feature = model.classifier[-1].in_features
        model.classifier[-1] = torch.nn.Linear(in_feature, 10)

    model.cuda()
    return model

In [ ]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
from models import get_model


def reset(cls):
    if cls == 'resnet':
        model = models.resnet18(pretrained=False)
        model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        model.fc = nn.Linear(model.fc.in_features, 10)
    elif cls == 'vgg':
        model = models.vgg13(pretrained=False)
        model.features[0] = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
        for i, layer in enumerate(model.features):
            if isinstance(layer, nn.MaxPool2d):
                model.features[i] = nn.MaxPool2d(kernel_size=2, stride=1, padding=1)
        model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, 10)
    elif cls == 'dense':
        model = models.densenet121(pretrained=False)
        model.features.conv0 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        model.classifier = nn.Linear(model.classifier.in_features, 10)
    elif cls == 'mobile':
        model = models.mobilenet_v2(pretrained=False)
        model.features[0][0] = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False)
        model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, 10)
    else:
        raise ValueError(f"Unknown model type: {cls}")
    return model


def load_model(num, mode):
    if mode == "source":
        path = "./pretrained_models/teacher_model.pth"
        model = get_model("lenet", "mnist", False)

    elif mode == "finetune":
        path = f"./trained_models/finetune/finetune_{num}.pth"
        model = get_model("lenet", "mnist", False)

    elif mode == "finetune_fragile":
        path = f"./trained_models/finetune/finetune_fragile_{num}.pth"
        model = get_model("lenet", "mnist", False)

    elif mode == "kd":
        path = f"./trained_models/kd/student_model_kd_{num}.pth"
        if num < 5:
            cls = 'vgg'
        elif num < 10:
            cls = 'resnet'
        elif num < 15:
            cls = 'dense'
        else:
            cls = 'mobile'
        model = reset(cls)

    elif mode == "clean":
        path = f"./trained_models/irrelevant/clean_model_{num}.pth"
        if num < 5:
            cls = 'vgg'
        elif num < 10:
            cls = 'resnet'
        elif num < 15:
            cls = 'dense'
        else:
            cls = 'mobile'
        model = reset(cls)

    elif mode == "prune":
        path = f"./trained_models/fine_pruning/prune_model_{num}.pth"
        model = torch.load(path)
        return model

    elif mode == "tl":
        path = f"./trained_models/transfer/tl{num}.pth"
        model = get_model("lenet", "mnist", False)

    else:
        raise ValueError(f"Unsupported mode: {mode}")

    model.load_state_dict(torch.load(path))
    return model

In [ ]:
models_list = []

for i in [0]:
    globals()['teacher' + str(i)] = load_model(i, "source")
    models_list.append(globals()['teacher' + str(i)])


for i in range(20):
    globals()['mep' + str(i)] = load_model(i, "kd") #MEP
    models_list.append(globals()['mep' + str(i)])

for i in range(20):
    globals()['mel' + str(i)] = load_model(i, "1")  #MEL
    models_list.append(globals()['mel' + str(i)])

for i in range(20):
    globals()['clean' + str(i)] = load_model(i, "clean")
    models_list.append(globals()['clean' + str(i)])

for i in range(5): #TL
    globals()['KMNIST' + str(i)] = load_model(i, "tl")
    models_list.append(globals()['KMNIST' + str(i)])


for i in range(15):
    globals()['fp' + str(i)] = load_model(i, "prune") #FP
    models_list.append(globals()['fp' + str(i)])


for i in range(20):
    globals()['finetune' + str(i)] = load_model(i, 'finetune') #0-10:FT-all, 10-20 FT-last
    models_list.append(globals()['finetune' + str(i)])




In [ ]:
#AUC
for i in range(1):
    iter = i
    print("Iter:", iter)
    cal_correlation(models_list,iter)

In [ ]:
#fragility Tamper detection rate
models_list = []

for i in [0]:
    globals()['teacher' + str(i)] = load_model(i, "source")
    models_list.append(globals()['teacher' + str(i)])


for i in range(20):
    globals()['mep' + str(i)] = load_model(i, "kd") #MEP
    models_list.append(globals()['mep' + str(i)])

for i in range(20):
    globals()['mel' + str(i)] = load_model(i, "1")  #MEL
    models_list.append(globals()['mel' + str(i)])


for i in range(5): #TL
    globals()['KMNIST' + str(i)] = load_model(i, "tl")
    models_list.append(globals()['KMNIST' + str(i)])


for i in range(16):
    globals()['fp' + str(i)] = load_model(i, "prune") #FP
    models_list.append(globals()['fp' + str(i)])


for i in range(20):
    globals()['finetune_fragile' + str(i)] = load_model(i, 'finetune_fragile') #FT-L epoch 0~100
    models_list.append(globals()['finetune_fragile' + str(i)])

for i in range(1):
        iter = i
        print("Iter:", iter)
        cal_fragility(models_list, iter)

In [ ]:
#Uni-hard_label
models_list = []

for i in [0]:
    globals()['teacher' + str(i)] = load_model(i, "source")
    models_list.append(globals()['teacher' + str(i)])


for i in range(20):
    globals()['clean' + str(i)] = load_model(i, "clean")
    models_list.append(globals()['clean' + str(i)])

    
for i in range(1):
        iter = i
        print("Iter:", iter)
        cal_acc(models_list, iter) #unic, similarity 

In [ ]:
#Uni-soft_labels
def extract_features(model, inputs, device='cuda'):
    model.eval()
    with torch.no_grad():
        inputs = inputs.to(device)
        outputs = model(inputs)
        outputs = F.softmax(outputs, dim=1)
    return outputs.cpu()


def distance_function(a, b):
    epsilon = 1e-9
    a = torch.clamp(a, epsilon, 1)
    b = torch.clamp(b, epsilon, 1)
    return (a * (a.log() - b.log())).sum(dim=1)


def estimate_uniqueness(source_model, fingerprint_dataset, unrelated_models, delta_values, device='cuda'):
    fingerprint_loader = DataLoader(fingerprint_dataset, batch_size=100, shuffle=False)
    all_fingerprints = torch.cat([x.to(device) for x, _ in fingerprint_loader], dim=0)

    fM = []
    with torch.no_grad():
        for i in range(0, len(all_fingerprints), 10):
            batch = all_fingerprints[i:i+10]
            features = extract_features(source_model, batch, device=device)
            fM.append(features)
    fM = torch.cat(fM, dim=0)

    results = []

    for delta in delta_values:
        min_distances = []
        avg_distances = []
        collision_count = 0
        for umodel in unrelated_models:
            fMprime = []
            with torch.no_grad():
                for i in range(0, len(all_fingerprints), 100):
                    batch = all_fingerprints[i:i+100]
                    features = extract_features(umodel, batch, device=device)
                    fMprime.append(features)
            fMprime = torch.cat(fMprime, dim=0)

            dist = distance_function(fM, fMprime)
            min_dist = dist.min().item()
            avg_dist = dist.mean().item()
            min_distances.append(min_dist)
            avg_distances.append(avg_dist)

            if torch.any(dist <= delta):
                collision_count += 1

        uniqueness = 1 - (collision_count / len(unrelated_models))
        results.append((delta.item(), uniqueness, min(min_distances), sum(avg_distances) / len(avg_distances)))
        print(f"Δ={delta:.2f} | Uniqueness={uniqueness:.4f} | MinDist={min(min_distances):.4f} | AvgDist={sum(avg_distances)/len(avg_distances):.4f}")

    return results


if __name__ == "__main__":
    device = 'cuda'
    delta_values = torch.arange(0, 0.11, 0.01)


    source_model = load_model(0, "source").to(device)
    source_model.eval()


    dataset_path = "./data/generate_DiffFP_boundary_100.h5"
    full_dataset = get_dataset(dataset_path)
    fingerprint_dataset = Subset(full_dataset, list(range(0, 200, 2)))  # only x_w

    unrelated_models = [load_model(i, "clean").to(device).eval() for i in range(20)]

    print("Evaluating Uniqueness...")
    results = estimate_uniqueness(source_model, fingerprint_dataset, unrelated_models, delta_values, device=device)

    print("\nFinal Summary:")
    for delta, uniq, min_dist, avg_dist in results:
        print(f"Δ={delta:.2f} | Uniqueness={uniq:.4f} | MinDist={min_dist:.4f} | AvgDist={avg_dist:.4f}")